In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs

In [32]:
def getLink(page):
    html=urlopen("https://movie.naver.com/movie/point/af/list.naver")
    obj=bs(html, 'html.parser')
    all_text=obj.select('td.title')
    #print(all_text)
    data_list=[]
    for td in all_text:
        text_arr=td.text.replace('신고','').replace("\t","").replace("\n\n\n\n","").strip().split("\n\n")
        text_arr[1]=int(text_arr[1][text_arr[1].find('중')+1:])
        if len(text_arr)<3:
            text_arr.append('-')
        data_list.append(tuple(text_arr))
    #print(data_list)
    return data_list

In [33]:
import pandas as pd
def csv_save(data_total):
    df=pd.DataFrame(data_total, columns=['title','point','review'])
    df.to_csv('data/movie_reviews.csv',index=False)
    #print(df)

In [36]:
import pymysql
def mysql_save(data_total):
    conn=pymysql.connect(host='localhost', user='nyy', password='1234', db='py_mysql_db', charset='utf8')
    cursor=conn.cursor()
    sql="insert into movie_reviews(title, point, review) values(%s, %s, %s)"
    cursor.executemany(sql, data_total)
    conn.commit()
    conn.close()

In [37]:
data_total=[]
for i in range(1,11):
    data_total.extend(getLink(i))
#print(data_total)
#csv_save(data_total)
mysql_save(data_total)

In [40]:

title_list=[]
point_list=[]
review_list=[]
movie_reviews=[]

titles=obj.select('td.title>a')
for i in range(len(titles)):
    if i%2==0:
        title=titles[i].string
        title_list.append(title)

points=obj.select('td.title > div > em')
#print(points)
for point in points:
    point_list.append(int(point.string))


reviews=obj.select('td.title')
for review in reviews:
    text_arr=review.text.replace('신고','').replace("\t","").replace("\n\n\n\n","").strip().split("\n\n")
    if len(text_arr)<3:
        review_list.append('-')
    else:
        review_list.append(text_arr[2])
        
#print(review_arr)

for i in range(10):
    movie_reviews.append(tuple([title_list[i],point_list[i],review_list[i]]))
print(movie_reviews)

[('싱크홀', 8, '걍 진짜 킬링타임용 앜엌 차승원 개웃겨 구세주임희생자 부분에서 좀 신파 감성이 없지 않아 있긴 함'), ('장르만 로맨스', 7, '배우들이 좋아서 봤어요. 그런대로 볼만해요.'), ('틱, 틱... 붐!', 10, '뮤지컬 영화는 항상 평균 이상은 한다. 음악과 함께 배우들의 율동과 노래로 관객들의 마음을 움직이기 때문이다. 여기에 실화 바탕의 스토리라면 이보다 더 완벽할 수가 없다.'), ('싱크홀', 4, '넷플에서 한 시간 정도 보다가 도저히 끝까지 볼 인내가 바닥나서 감독정보를 보니.. 음.. 7광구를 만드셨던 분이네요. 그래도 화려한 휴가는 괜찮았는데.. 다른 건 둘째치고 앞으로 유머코드는 시도 안하셨음 좋겠어요. 너무 감떨어지고 유치해요. 요즘 액션, CG가  중요하다 하지만 스토리와 대사발도 여전히 중요하답니다. 영화에 철학까진 안담겨도 의미를 주거나 재미를 주거나 아님 감동이 있거나 셋 중 하나는 있어야죠. 그래도 100만 넘게 봤다해서 킬링타임용으로 보려했는데 waste time 이 될 것 같아 보다 껐네요.'), ('연애 빠진 로맨스', 8, '내 나이 29살 결말  빼고 완벽했다 내 결말인가 싶었다'), ('샹치와 텐 링즈의 전설', 10, '샹치보단 양조위가 개간지다 ㄷㄷ'), ('이터널스', 1, '희대의 쓰레기영화 내용을 신경쓴게 아니고 pc주의라는 이상한 싸이비 이념을 강조...'), ('이터널스', 1, '히로시마 원폭이 뭐 대학살? 일본이 피해자라고?각본가 너때문에 이제 마블 거른다'), ('레버넌트: 죽음에서 돌아온 자', 9, '일종의 자연재해처럼 그려지는 인디언 원주민들로 인해 복수가 배경이고 자연이 주인공처럼 느껴진다.'), ('유체이탈자', 10, '스릴넘치고 굉장한 영화네요!')]
